load csv from downloads
normalize and rename
split train/test
try logr and gbc baseline
evaluate on accuracy (imbalanced?) -goal is 65%+

try pytorch 2 layer

In [1]:
import numpy as np
import pandas as pd

## Data Prep

In [2]:
df_pi_raw = pd.read_csv(r"C:\Users\Devan\Downloads\datasets\pima-indians-diabetes.csv")
#print(df_pi_raw.info())

Number of times pregnant. <br>
Plasma glucose concentration a 2 hours in an oral glucose tolerance test. 
Diastolic blood pressure (mm Hg).
Triceps skinfold thickness (mm).
2-Hour serum insulin (mu U/ml).
Body mass index (weight in kg/(height in m)^2).
Diabetes pedigree function.
Age (years).
Class variable (0 or 1).

In [3]:
cols = ['pregnant_count','plasma_gluc','bp','skinfold','insulin','bmi','pedigree','age','y_class']
df_pi_raw.set_axis(cols, axis=1, inplace=True)
#print(df_pi_raw.info())

In [4]:
#standardize data
from sklearn.preprocessing import StandardScaler
df_pi_X_std = pd.DataFrame(index=df_pi_raw.index)
for c in df_pi_raw.columns:
    if c != 'y_class':
        new_col = c+'_std'
        df_pi_X_std[new_col] = StandardScaler().fit_transform(df_pi_raw.loc[:,[c]])
df_pi_Y = df_pi_raw.loc[:,['y_class']]

In [5]:
#train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_pi_X_std, df_pi_Y
                                                    ,train_size=.8
                                                    ,test_size=.2
                                                    ,stratify= df_pi_Y
                                                    ,random_state=123
                                                   )

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 560 to 354
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pregnant_count_std  613 non-null    float64
 1   plasma_gluc_std     613 non-null    float64
 2   bp_std              613 non-null    float64
 3   skinfold_std        613 non-null    float64
 4   insulin_std         613 non-null    float64
 5   bmi_std             613 non-null    float64
 6   pedigree_std        613 non-null    float64
 7   age_std             613 non-null    float64
dtypes: float64(8)
memory usage: 59.3 KB


## LogR and GBC

In [61]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression()
logr.fit(X_train,np.ravel(y_train))

LogisticRegression()

In [80]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=50,max_depth=5)
gbc.fit(X_train,np.ravel(y_train))

GradientBoostingClassifier(max_depth=5, n_estimators=50)

In [100]:
#Evaluation on Test
from sklearn.metrics import f1_score
print(f'Logr Acc score:  {logr.score(X_test,y_test):0.2f}')
print(f'GBC Acc score:   {gbc.score(X_test,y_test):0.2f}')
print(f'Logr F1 score:   {f1_score(y_test, logr.predict(X_test)):0.2f}')
print(f'GBC F1 score:    {f1_score(y_test, gbc.predict(X_test)):0.2f}')
logrf1_w = f1_score(y_test, logr.predict(X_test), average='weighted')
gbcf1_w = f1_score(y_test, gbc.predict(X_test), average='weighted')
print(f'Logr F1 w score: {logrf1_w:0.2f}')
print(f'GBC F1 w score:  {gbcf1_w:0.2f}')

Logr Acc score:  0.81
GBC Acc score:   0.74
Logr F1 score:   0.69
GBC F1 score:    0.60
Logr F1 w score: 0.80
GBC F1 w score:  0.73


## Pytorch

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
#LOGR test
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(8, 2)

    def forward(self, x):
        x = F.sigmoid(self.l1(x))
        return x

net = Net()

In [38]:
#Deep Net v1
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(8, 4)
        #self.h1 = nn.Linear(6, 4)
        self.o1 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.l1(x))
        #x = F.relu(self.h1(x))  #relu too high, sig too low
        #x = self.o1(x)  #ok by self
        #x = F.softmax(self.o1(x))
        x = F.sigmoid(self.o1(x))
        return x

net = Net()

In [39]:
criterion = nn.CrossEntropyLoss()
#criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [30]:
# recreate data as a tensor tbd
#torch.tensor(np.ravel(X_train.loc[111]))
#torch.tensor(np.ravel(y_train.loc[111]+0.))
#criterion(torch.tensor(.444), torch.tensor(1.0))
#torch.tensor(np.ravel([y_train.loc[111]+0.,1-y_train.loc[111]+0.]))

p = torch.tensor([.4778])
t = torch.tensor([1.0])
p = torch.tensor([.4778,1.0-.4778])
t = torch.tensor([1.0, 1.0-1.0])
criterion(p, t)
#F.binary_cross_entropy(torch.tensor([.477,1-.4777]), torch.tensor([1.0,0.0]))

tensor(0.7156)

In [40]:
#training
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    i = 0
    #for i, data in enumerate(trainloader, 0):
    for r in X_train.index:
        ## get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs = torch.tensor(np.ravel(X_train.loc[r]))
        labels = torch.tensor(np.ravel([y_train.loc[r]+0.,1.0-y_train.loc[r]+0.]))  #converted to onehot
        #labels = torch.tensor(np.ravel(y_train.loc[r]+0.))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        #loss = F.binary_cross_entropy(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
            #print(f'{outputs=} // {labels=}')
        i = i+1

print(net(torch.tensor(np.ravel(X_test.iloc[1])).float()))

[1,   200] loss: 0.669
outputs=tensor([0.3990, 0.5901], grad_fn=<SigmoidBackward0>) // labels=tensor([0., 1.], dtype=torch.float64)
[1,   400] loss: 0.670
outputs=tensor([0.3611, 0.5850], grad_fn=<SigmoidBackward0>) // labels=tensor([0., 1.], dtype=torch.float64)
[1,   600] loss: 0.660
outputs=tensor([0.6164, 0.7218], grad_fn=<SigmoidBackward0>) // labels=tensor([1., 0.], dtype=torch.float64)
[2,   200] loss: 0.655
outputs=tensor([0.3601, 0.6459], grad_fn=<SigmoidBackward0>) // labels=tensor([0., 1.], dtype=torch.float64)
[2,   400] loss: 0.659
outputs=tensor([0.3244, 0.6601], grad_fn=<SigmoidBackward0>) // labels=tensor([0., 1.], dtype=torch.float64)
[2,   600] loss: 0.648
outputs=tensor([0.6451, 0.7234], grad_fn=<SigmoidBackward0>) // labels=tensor([1., 0.], dtype=torch.float64)
[3,   200] loss: 0.641
outputs=tensor([0.3250, 0.7010], grad_fn=<SigmoidBackward0>) // labels=tensor([0., 1.], dtype=torch.float64)
[3,   400] loss: 0.648
outputs=tensor([0.2848, 0.7447], grad_fn=<SigmoidBack

In [41]:
#testing
pred_out = []
print(net(torch.tensor(np.ravel(X_test.iloc[1])).float()))
for r in X_test.index:
    inputs = torch.tensor(np.ravel(X_test.loc[r]))
    outputs = net(inputs.float())
    #print(outputs)
    if outputs[1] > outputs[0]:
        pred_out.append(1)
    else:
        pred_out.append(0)

for i in range(10):
   print("Actual {}\tPredicted: {}".format(y_test.iloc[i].item(), pred_out[i]))

print(y_test.sum().item()) #54 of 154
print(pd.DataFrame(pred_out).sum().item())

tensor([0.0698, 0.9439], grad_fn=<SigmoidBackward0>)
Actual 1	Predicted: 1
Actual 0	Predicted: 1
Actual 1	Predicted: 0
Actual 0	Predicted: 1
Actual 0	Predicted: 0
Actual 0	Predicted: 1
Actual 1	Predicted: 1
Actual 0	Predicted: 1
Actual 0	Predicted: 1
Actual 1	Predicted: 1
54
119


## SHAP

In [ ]:
#feature interpretation
#package install - tbd